In [72]:
import pandas as pd
import re
import numpy as np

In [73]:
normal = pd.read_excel('COMBINED_NORMAL_ADVANCED_STATS_1984-85_to_2019-20_ver2.xlsx', sheet_name=0)

In [78]:
normal = normal[['slug', 'name', 'positions', 'age', 'team', 'games_played',
       'games_started', 'minutes_played', 'made_field_goals',
       'attempted_field_goals', 'made_three_point_field_goals',
       'attempted_three_point_field_goals', 'made_free_throws',
       'attempted_free_throws', 'offensive_rebounds', 'defensive_rebounds',
       'assists', 'steals', 'blocks', 'turnovers', 'personal_fouls', 'points', 'Year']]

In [87]:
normal['end season'] = normal['Year'].apply(lambda x: '19' + x[-2:] if int(x[-2:]) >= 84 else '20' + x[-2:])
normal['key'] = normal['name'] + normal['end season']

<ipython-input-87-2a801766409e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  normal['end season'] = normal['Year'].apply(lambda x: '19' + x[-2:] if int(x[-2:]) >= 84 else '20' + x[-2:])
<ipython-input-87-2a801766409e>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  normal['key'] = normal['name'] + normal['end season']


In [120]:
normal = normal.drop(columns=['slug', 'Year'])

In [133]:
normal_19_20 = pd.read_excel('COMBINED_NORMAL_ADVANCED_STATS_1984-85_to_2019-20_ver2.xlsx', sheet_name=3)

In [134]:
normal_19_20['Player'] = normal_19_20['Player'].apply(lambda x: re.sub(r"\\.+", "", x))

In [142]:
columns_change = dict(zip(['Player', 'Pos', 'Age', 'Tm', 'G', 'GS', 'MP', 'FG', 'FGA',
       '3P', '3PA', 'FT', 'FTA',
       'ORB', 'DRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS'],
                          ['name', 'positions', 'age', 'team', 'games_played',
       'games_started', 'minutes_played', 'made_field_goals',
       'attempted_field_goals', 'made_three_point_field_goals',
       'attempted_three_point_field_goals', 'made_free_throws',
       'attempted_free_throws', 'offensive_rebounds', 'defensive_rebounds',
       'assists', 'steals', 'blocks', 'turnovers', 'personal_fouls', 'points']))

In [143]:
normal_19_20 = normal_19_20.rename(columns=columns_change)
normal_19_20 = normal_19_20.drop(columns=['3P%', '2P', '2PA', '2P%', 'eFG%', 'FG%', 'FT%', 'TRB', 'Rk'])
normal_19_20['end season'] = '2020'
normal_19_20['key'] = normal_19_20['name'] + normal_19_20['end season']

In [155]:
normal_agg = normal.append(normal_19_20)

In [184]:
normal_agg = normal_agg.reset_index(drop=True)

In [147]:
stats = pd.read_excel('COMBINED_NORMAL_ADVANCED_STATS_1984-85_to_2019-20_ver2.xlsx', sheet_name=1)

In [148]:
stats['error_in_name'] = stats['slug'].apply(lambda x: x is np.nan)

In [149]:
stats_need_fix = stats.loc[stats['error_in_name'] == True]

In [150]:
stats_need_fix['Year'] = '2019-20'
name = stats_need_fix['name'].apply(lambda x: re.sub(r"\\.+", "", x))
slug = stats_need_fix['name'].apply(lambda x: re.sub(r".+\\", "", x))
stats_need_fix['name'], stats_need_fix['slug'] = name, slug

<ipython-input-150-5b3e32999086>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stats_need_fix['Year'] = '2019-20'
<ipython-input-150-5b3e32999086>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stats_need_fix['name'], stats_need_fix['slug'] = name, slug


In [151]:
stats = stats.loc[stats['error_in_name'] == False].append(stats_need_fix).drop(columns=['error_in_name'])

In [152]:
stats['end season'] = stats['Year'].apply(lambda x: '19' + x[-2:] if int(x[-2:]) >= 84 else '20' + x[-2:])

In [153]:
stats['key'] = stats['name'] + stats['end season']

In [186]:
stats = stats.reset_index(drop=True)

In [195]:
stats_agg = pd.concat([normal_agg.drop(columns=['positions', 'team','key']),stats.drop(columns=['name', 'positions',
       'age', 'team', 'games_played', 'minutes_played','Year','slug','end season','is_combined_totals'])], axis=1)

In [196]:
salaries = pd.read_excel('salaries_1985to2020_final.xlsx')
salary_cap = pd.read_excel('salary_caps.xlsx')

In [197]:
merged = salaries.merge(salary_cap[['season', 'salary_cap']], on='season')

In [198]:
merged['cap_space_usage'] = merged['salary'] / merged['salary_cap']

In [199]:
merged['season_end'] = merged['season_end'].apply(lambda x: str(x))

In [200]:
merged['key'] = merged['names'] + merged['season_end']

In [201]:
final_table = stats_agg.merge(merged, on='key').drop(columns=[])

In [203]:
final_table.to_csv('bbr_normal_advanced_cap_usage.csv')